In [28]:
#------------------------------------------------------------------------
# WES 269 SBUS Demo Code
#
# This code serves as a baseline for configuring the UART16550 core
# and polling data from the RX FIFO
#------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

In [29]:
# Load bitstream
ol = Overlay('SBUS_DEMO_ext_clk_inverter.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

  
0.7480342388153076 seconds to program bitstream


In [30]:
# Address space definitions
IP_BASE_ADDRESS = 0X43C00000
ADDRESS_RANGE = 0x10000

RX_BUF_REG       =   0x1000        #Receiver Buffer Register              LCR[7] = 0
TX_HOLD_REG      =   0x1000        #Transmitter Holding Register          LCR[7] = 0
INT_EN_REG       =   0x1004        #Interrupt Enable Register             LCR[7] = 0
INT_ID_REG       =   0x1008        #Interrupt Identification Register     LCR[7] = x
FIFO_CNTRL_REG   =   0x1008        #FIFO Control Register                 LCR[7] = x (write), LCR[7] = 1 (read)
LINE_CNTRL_REG   =   0x100C        #Line Control Register                 LCR[7] = x
MODEM_CNTRL_REG  =   0x1010        #Modem Control Register                LCR[7] = x
LINE_STATUS_REG  =   0x1014        #Line Status Register                  LCR[7] = x
MODEM_STATUS_REG =   0x1018        #Modem Status Register                 LCR[7] = x
SCRATCH_REG      =   0x101C        #Scratch Register                      LCR[7] = x
DIV_LATCH_LSB    =   0x1000        #Divisor Latch LSB Register            LCR[7] = 1
DIV_LATCH_MSB    =   0x1004        #Divisor Latch MSB Register            LCR[7] = 1

NUM_BYTES        =   25
START_BYTE     =   0xF0
STOP_BYTE      =   0x00

In [31]:
# configuring UART16550 core

# enable divisor latch, set baud divisor
mmio.write(LINE_CNTRL_REG, 0x80)
#mmio.write(DIV_LATCH_LSB, 0x3E) # divide by 62 for 100MHz AXI clock
mmio.write(DIV_LATCH_LSB, 0x1) # divide by 1 for external 1.6 MHz BAUD rate clock
mmio.write(DIV_LATCH_MSB, 0x0)

# configure data exchange, 100k BAUD, even parity, 2 stop bits, 8 data bits
mmio.write(LINE_CNTRL_REG, 0x1F)

# enable and configure FIFO
mmio.write(FIFO_CNTRL_REG, 0x1)

# reset FIFOs
mmio.write(FIFO_CNTRL_REG, 0x7)
mmio.write(FIFO_CNTRL_REG, 0x1)


print('UART16550 Core Configuration Complete')

UART16550 Core Configuration Complete


In [45]:
# reset FIFOs

mmio.write(FIFO_CNTRL_REG, 0x7)
#mmio.write(FIFO_CNTRL_REG, 0x1)


In [46]:
# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
BYTES_READ = 0


while(BYTES_READ != (NUM_BYTES)):
    
    #read from the RX FIFO when data present
    while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        pass
    
    buffer[0] = mmio.read(RX_BUF_REG)
    if ( buffer[0] == 0xF0 ):
        BYTES_READ += 1
        for i in range(NUM_BYTES - 1):
            buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
            BYTES_READ += 1
    else:
        pass
    print('Total bytes read: ', BYTES_READ)
    
print(''.join('%02x'%i for i in buffer))   # printing buffer contents


Total bytes read:  0
Total bytes read:  0
Total bytes read:  0
Total bytes read:  0
Total bytes read:  0
Total bytes read:  25
f02322ee2ea3ac24021020f002020202020202020202020202


In [47]:
# parsing buffered packet into channel data

# channel 1   Throttle
# channel 2   Roll
# channel 3   Pitch
# channel 4   yaw

NUM_CHANNELS = 4

channel = [0]*NUM_CHANNELS

# bit masking to parse 11 bit channel data from buffer
channel[0] = (buffer[1]      | buffer[2] << 8)                    & 0x7FF
channel[1] = (buffer[2] >> 3 | buffer[3] << 5)                    & 0x7FF
channel[2] = (buffer[3] >> 6 | buffer[4] << 2 | buffer[5] << 10)  & 0x7FF
channel[3] = (buffer[5] >> 1 | buffer[6] << 7)                    & 0x7FF


print('Throttle Command:  ', channel[0])
print('Roll Command:      ', channel[1])
print('Pitch Command:     ', channel[2])
print('Yaw Command:       ', channel[3])



Throttle Command:   547
Roll Command:       1476
Pitch Command:      1211
Yaw Command:        1617


In [35]:
#########################################
# old code
#########################################

# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
#buffer = [0]*(NUM_BYTES-1)
BYTES_READ = 0

test = [0]*(NUM_BYTES)


while(BYTES_READ != (NUM_BYTES)):
#while(BYTES_READ != (NUM_BYTES - 1)):
    
    #read from the RX FIFO when data present
    #while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        #pass
    
    #print(mmio.read(RX_BUF_REG))
    #print(idx)
    
    #if(mmio.read(RX_BUF_REG) != 240):
        #pass
        #print('hard pass')
        
        
    for index in range(NUM_BYTES):
        buffer[index] = mmio.read(RX_BUF_REG)
        BYTES_READ += 1
        
    #temp = mmio.read(RX_BUF_REG)
    #if (temp != 240):
    #    print(temp)
    #else:
    #    buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
    #    BYTES_READ += 1

print(buffer) 
print('')
print(''.join('%02x'%i for i in buffer))
print('')

for i in range(NUM_BYTES):
    test[i] = ~buffer[i]
    
print(test)
print('')
print(''.join('%02x'%i for i in test))

[163, 43, 246, 46, 163, 172, 36, 2, 16, 32, 2, 16, 32, 2, 240, 163, 16, 32, 2, 46, 46, 46, 46, 16, 163]

a32bf62ea3ac2402102002102002f0a31020022e2e2e2e10a3

[-164, -44, -247, -47, -164, -173, -37, -3, -17, -33, -3, -17, -33, -3, -241, -164, -17, -33, -3, -47, -47, -47, -47, -17, -164]

-a4-2c-f7-2f-a4-ad-25-3-11-21-3-11-21-3-f1-a4-11-21-3-2f-2f-2f-2f-11-a4


In [36]:
# testing grabbing channel 1

print('buffer[1]:             ', buffer[1])
print('buffer[2]:             ', buffer[2])
print('buffer[2] last 3 bits: ', buffer[2] & 0x07)

temp = buffer[1] | (buffer[2] << 8)
temp2 = temp & 0x07FF
temp3 = (buffer[1] | buffer[2] << 8) & 0x07FF
 
print('temp:                  ', temp)
print('temp2:                 ', temp2)
print('temp3:                 ', temp3)

start = buffer[0]
print('start byte:            ', start)
channel_1 = (buffer[1] | ( buffer[2] << 8 )) & 0x07FF

print('channel 1:             ', channel_1)

buffer[1]:              43
buffer[2]:              246
buffer[2] last 3 bits:  6
temp:                   63019
temp2:                  1579
temp3:                  1579
start byte:             163
channel 1:              1579


In [37]:
# TRAVIS IDEAS for infinite graph display

while !quit:
  get packet
  update the graph
  send stuff if the user wants to send stuff
    i.e. break commands

SyntaxError: invalid syntax (<ipython-input-37-a438acf29402>, line 3)